In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 21.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import global_add_pool
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import GIN_MLP_Layers as gin_mlp_layers
from scipy.sparse import csr_matrix
from sklearn import metrics
from time import perf_counter

In [ ]:
'''class GIN_MLPs(nn.Module):
    def __init__(self, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, Bias):
        super(GIN_MLPs, self).__init__()
        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.Bias = Bias

        if mlp_act_fun == 'ReLu':
            self.mlp_act_fun = F.relu
        elif mlp_act_fun == 'eLu':
            self.mlp_act_fun = nn.functional.elu
        elif mlp_act_fun == 'tanh':
            self.mlp_act_fun = torch.tanh

        self.gin_mlp_layers = torch.nn.ModuleList()
        self.gin_batch_normalization = torch.nn.ModuleList()


        if self.num_slp_layers == 1:
            self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_output_dim, bias=self.Bias))
            self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_output_dim))
        elif self.num_slp_layers > 1:
            for i in range(self.num_slp_layers):
                if i == 0:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))
                elif i == self.num_slp_layers-1:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_output_dim, bias=self.Bias))
                    #self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_output_dim))
                elif 0 < i < self.num_slp_layers-1:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    #self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))
        else:
            print("please enter layer config")

    def forward(self, h):

        for i in range(self.num_slp_layers):
            if i == 0:
                layer = self.gin_mlp_layers[i](h)
                #layer = layer.permute(0, 2, 1)
                bnorm = self.gin_batch_normalization[i](layer)
                #bnorm = bnorm.permute(0, 2, 1)
                h = self.mlp_act_fun(bnorm)
            else:
                h = self.mlp_act_fun(self.gin_mlp_layers[i](h))
        return h
'''

In [16]:
class GIN_MLPs(nn.Module):
    def __init__(self, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, Bias):
        super(GIN_MLPs, self).__init__()
        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.Bias = Bias

        if mlp_act_fun == 'ReLu':
            self.mlp_act_fun = F.relu
        elif mlp_act_fun == 'eLu':
            self.mlp_act_fun = nn.functional.elu
        elif mlp_act_fun == 'tanh':
            self.mlp_act_fun = torch.tanh

        self.gin_mlp_layers = torch.nn.ModuleList()
        self.gin_batch_normalization = torch.nn.ModuleList()


        if self.num_slp_layers == 1:
            self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_output_dim, bias=self.Bias))
            self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_output_dim))
        elif self.num_slp_layers > 1:
            for i in range(self.num_slp_layers):
                if i == 0:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(num_features=self.mlp_hid_dim))
                elif i == self.num_slp_layers-1:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_output_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_output_dim))
                elif 0 < i < self.num_slp_layers-1:
                    self.gin_mlp_layers.append(nn.Linear(in_features=self.mlp_hid_dim, out_features=self.mlp_hid_dim, bias=self.Bias))
                    self.gin_batch_normalization.append(nn.BatchNorm1d(self.mlp_hid_dim))
        else:
            print("please enter layer config")

    def forward(self, h):

        for i in range(self.num_slp_layers):
            if i == 0:
                layer = self.gin_mlp_layers[i](h)
                #layer = layer.permute(0, 2, 1)
                bnorm = self.gin_batch_normalization[i](layer)
                #bnorm = bnorm.permute(0, 2, 1)
                h = self.mlp_act_fun(bnorm)
            else:
                h = self.mlp_act_fun(self.gin_mlp_layers[i](h))
        return h


mlp_example = GIN_MLPs(num_slp_layers=2, mlp_input_dim=7, mlp_hid_dim=7, mlp_output_dim=7, mlp_act_fun="ReLu", Bias=True)
x = torch.rand(20,7)
#x = x.permute(0, 2, 1)
y = mlp_example(x)
print(y.size())

torch.Size([20, 7])


In [ ]:
# With Learnable Parameters
#m = nn.BatchNorm1d(100)
# Without Learnable Parameters
m = nn.BatchNorm1d(7, affine=False)
input = torch.randn(300, 100, 7)
input = input.permute(0, 2, 1)
output = m(input)
output = output.permute(0, 2, 1)
print(output.size())

torch.Size([300, 100, 7])


In [ ]:
class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################

In [ ]:
global_sum_pool = GlobalSUMPool()

In [ ]:
x = torch.arange(140)
x = x.view(2,10, 7)
print(x)
x = x.permute()
print(x)

tensor([[[  0,   1,   2,   3,   4,   5,   6],
         [  7,   8,   9,  10,  11,  12,  13],
         [ 14,  15,  16,  17,  18,  19,  20],
         [ 21,  22,  23,  24,  25,  26,  27],
         [ 28,  29,  30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39,  40,  41],
         [ 42,  43,  44,  45,  46,  47,  48],
         [ 49,  50,  51,  52,  53,  54,  55],
         [ 56,  57,  58,  59,  60,  61,  62],
         [ 63,  64,  65,  66,  67,  68,  69]],

        [[ 70,  71,  72,  73,  74,  75,  76],
         [ 77,  78,  79,  80,  81,  82,  83],
         [ 84,  85,  86,  87,  88,  89,  90],
         [ 91,  92,  93,  94,  95,  96,  97],
         [ 98,  99, 100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109, 110, 111],
         [112, 113, 114, 115, 116, 117, 118],
         [119, 120, 121, 122, 123, 124, 125],
         [126, 127, 128, 129, 130, 131, 132],
         [133, 134, 135, 136, 137, 138, 139]]])


RuntimeError: ignored

In [106]:
from IPython.core.display import deepcopy
class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GIN_Model(nn.Module):
    def __init__(self, num_mlp_layers, Bias, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, num_classes,
                 dropout_rate, Weight_Initializer):
        super(GIN_Model, self).__init__()

        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.mlp_act_fun = mlp_act_fun
        self.lin_act_fun = mlp_act_fun
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.Weight_Initializer = Weight_Initializer

        self.num_mlp_layers = num_mlp_layers
        self.Bias = Bias

        self.eps = nn.Parameter(torch.zeros(self.num_mlp_layers),requires_grad=True)
        self.gin_mlp_layers = nn.ModuleList()
        self.global_summing = GlobalSUMPool()

        self.lin1 = nn.Linear(in_features=self.mlp_output_dim * (self.num_mlp_layers + 1), out_features=self.mlp_input_dim * (self.num_mlp_layers + 1))
        self.lin2 = nn.Linear(in_features=self.mlp_input_dim * (self.num_mlp_layers + 1), out_features=self.num_classes)
        self.dorpout = nn.Dropout(p=dropout_rate)
        self.act_fun_softmax = F.softmax
        self.the_first_layer = nn.Linear(in_features=self.mlp_input_dim, out_features=self.mlp_output_dim, bias=self.Bias)

        for i in range(self.num_mlp_layers):
            self.gin_mlp_layers.append(GIN_MLPs(num_slp_layers=self.num_slp_layers, mlp_input_dim=self.mlp_output_dim,
                                                               mlp_hid_dim=self.mlp_hid_dim, mlp_output_dim=self.mlp_output_dim,
                                                               mlp_act_fun=self.mlp_act_fun, Bias=self.Bias))

        if self.lin_act_fun == 'ReLu':
            self.lin_act_fun = F.relu
            #print('ReLu is Selected.')
        elif self.lin_act_fun == 'eLu':
            self.lin_act_fun = nn.functional.elu
            #print('eLu is Selected.')
        elif self.lin_act_fun == 'tanh':
            self.lin_act_fun = torch.tanh
            #print('tanh is Selected.')

        mean = 0
        std = 0.1
        self.initialize_weights(self.Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.xavier_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.xavier_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.kaiming_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.kaiming_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.normal_(modules.weight, mean=mean, std=std)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass


    def gin_neighborhood_aggregation(self, new_adjacecny, new_features):
        pooled = torch.mm(new_adjacecny, new_features)
        return pooled


    def gin_layer_process_eps(self, hid_rep, layer, new_adjacecny):

        pooled = self.gin_neighborhood_aggregation(new_adjacecny, hid_rep)
        pooled = pooled + (1 + self.eps[layer])*hid_rep
        pooled_rep = self.gin_mlp_layers[layer](pooled)

        return pooled_rep

    def merging_process(self, one_mlp, graph_sizes):
        new=[]
        start=0
        for j in range(len(graph_sizes)):
            end = start + graph_sizes[j]
            new.append(one_mlp[start:end])
            start = end
        return new

    def reshape_mlps_outputs(self, mlps_output_embeds, graph_sizes):
        merged_mlps_output_embeds = []
        for i in range(len(graph_sizes)):
            merged_mlps_output_embeds.append([])

        for i in range(len(mlps_output_embeds)):
            for j in range(len(mlps_output_embeds[i])):
                merged_mlps_output_embeds[j].extend(mlps_output_embeds[i][j])
        return merged_mlps_output_embeds

    def computational_matrix(self, batched_graphs, edge_mask):

        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())
        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)

        return joint_tilda_adjacency_matrix

    def reshape_batchs_by_graph_sizes(self, hid_rep, batch_size, graph_sizes):
        batch_mlps_output_embeds_shape_saved = []
        start = 0
        for i in range(batch_size):
            end = start + graph_sizes[i]
            un_padded_feat = hid_rep[start:end, start:end]#.unsqueeze(0)
            batch_mlps_output_embeds_shape_saved.append(un_padded_feat)
            start = end
        return batch_mlps_output_embeds_shape_saved

    def forward(self, batched_graphs, edge_mask):

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
            batch_size = batched_graphs.num_graphs
        else:
            graph_sizes = [len(batched_graphs.x)]
            batch_size = 1

        mlps_output_embeds = []
        mlps_output_embeds_shape_saved = []

        new_adjacecny = self.computational_matrix(batched_graphs, edge_mask)
        hid_rep = self.the_first_layer(batched_graphs.x)
        mlps_output_embeds.append(hid_rep)

        for layer in range(self.num_mlp_layers):
            hid_rep = self.gin_layer_process_eps(hid_rep, layer, new_adjacecny)
            mlps_output_embeds.append(hid_rep)
            mlps_output_embeds_shape_saved.append(self.reshape_batchs_by_graph_sizes(hid_rep, batch_size, graph_sizes))

        mlps_output_embeds_stacked = torch.stack(mlps_output_embeds)
        mlp_outputs_globalSUMpooled = self.global_summing(mlps_output_embeds_stacked, batched_graphs.batch)

        if batched_graphs.batch == None:
            merged_mlps_output_embeds_reshaped = torch.unsqueeze(mlp_outputs_globalSUMpooled, dim=1).view(1, -1)
        else:
            merged_mlps_output_embeds_reshaped = mlp_outputs_globalSUMpooled.view(mlp_outputs_globalSUMpooled.size()[1], -1)

        lin1_output = self.lin1(merged_mlps_output_embeds_reshaped)

        lin1_output = self.lin_act_fun(lin1_output)

        lin1_output_dropouted = self.dorpout(lin1_output)

        lin2_output = self.lin2(lin1_output_dropouted)
        lin2_output_softmaxed = self.act_fun_softmax(lin2_output, dim=1)

        return mlps_output_embeds_shape_saved, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed

node_feat_size=7
gin_model_example = GIN_Model(num_mlp_layers=4, Bias=False, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=17,
                              mlp_output_dim=19, mlp_act_fun="ReLu", num_classes=2, dropout_rate=0.5, Weight_Initializer=3)


In [80]:
dataset = TUDataset(root='data/TUDataset', name='MUTAG')
batch_size = 11
node_feat_size = len(dataset[0].x[0])
batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [9]:
GNN_Model_optimizer = torch.optim.Adam(gin_model_example.parameters(), lr=0.01, weight_decay=1e-4)

In [108]:
for batched_graphs in batched_dataset:
    #x, edge_index, batch, y = batched_graphs.x, batched_graphs.edge_index, batched_graphs.batch, batched_graphs.y
    mlps_output_embeds, mlps_output_embeds_stacked, mlps_output_embeds_pooled, mlps_output_embeds_pooled_stacked, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed = gin_model_example(batched_graphs, None)
    print("lin2_output_softmaxed: ", lin2_output_softmaxed.size())
    print(lin2_output_softmaxed)
    break

mlp_outputs_globalSUMpooled:  torch.Size([5, 11, 19])
merged_mlps_output_embeds_reshaped:  torch.Size([11, 95])
lin2_output_softmaxed:  torch.Size([11, 2])
tensor([[3.3732e-01, 6.6268e-01],
        [6.7942e-01, 3.2058e-01],
        [8.6440e-01, 1.3560e-01],
        [8.5527e-01, 1.4473e-01],
        [9.5747e-01, 4.2531e-02],
        [1.1380e-04, 9.9989e-01],
        [1.9320e-01, 8.0680e-01],
        [3.4980e-01, 6.5020e-01],
        [4.0564e-01, 5.9436e-01],
        [5.2087e-01, 4.7913e-01],
        [4.2795e-02, 9.5721e-01]], grad_fn=<SoftmaxBackward0>)


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
def loss_calculations(preds, gtruth):
    loss_per_epoch = criterion(preds, gtruth)
    return loss_per_epoch


In [ ]:
def train_step():
    GNN_Model_loss_batch = []
    Pred_Labels = []
    Real_Labels = []

    gin_model_example.train()
    gin_model_example.zero_grad()
    #torch.autograd.set_detect_anomaly(True)
    for batch_of_graphs in batched_dataset:

        mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, lin1_output_dropouted, lin2_output, softmaxed_h2 = gin_model_example(batch_of_graphs, None)
        batch_loss = loss_calculations(softmaxed_h2, batch_of_graphs.y)

        #print(softmaxed_h2, softmaxed_h2.argmax(dim=1))
        Pred_Labels.extend(softmaxed_h2.argmax(dim=1).detach().tolist())
        Real_Labels.extend(batch_of_graphs.y.detach().tolist())
        GNN_Model_loss_batch.append(batch_loss)

        batch_loss.backward()
        #print(gin_model_example.state_dict()['gin_mlp_layers.0.gin_mlp_layers.0.weight'])
        GNN_Model_optimizer.step()

    return torch.mean(torch.tensor(GNN_Model_loss_batch)), metrics.accuracy_score(Real_Labels, Pred_Labels)


In [ ]:
#from IPython.display import Javascript  # Restrict height of output cell.
#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))


SA_Model_training_time_per_epoch = []
SA_Model_training_Acc_per_epoch = []

def train(EPOCHS, load_index):
    SA_training_loss_per_epoch = []

    for epoch in range(EPOCHS):

        start_generation = perf_counter()
        SA_model_training_loss, training_acc = train_step()
        SA_Model_training_time_per_epoch.append(perf_counter() - start_generation)
        SA_Model_training_Acc_per_epoch.append(training_acc)

        print(f'Epoch: {epoch+1:03d}, Model Loss: {SA_model_training_loss:.4f} Accuracy: {training_acc}')

        SA_training_loss_per_epoch.append(SA_model_training_loss)


        #if (epoch + load_index + 1) % Visualization_Parameter == 0 and epoch > 0:
        #    visualize_losses(SA_training_loss_per_epoch, epoch + load_index + 1)
        #if (epoch + load_index + 1) % Model_Saving_Parameter == 0 and epoch > 0:
        #    torch.save({'epoch': epoch+load_index+1, 'model_state_dict': GNN_Model.state_dict(), 'optimizer_state_dict': GNN_Model_optimizer.state_dict(), 'loss': SA_training_loss_per_epoch,}, "/content/drive/My Drive/Explainability Methods/" + str(Explainability_name) + " on " + str(Task_name) + "/Model/" + File_Name + str(epoch + load_index + 1)+".pt")





In [ ]:
print(gin_model_example.state_dict()['gin_mlp_layers.0.gin_mlp_layers.0.weight'])

tensor([[-0.1422, -0.2985,  0.0476,  0.0016, -0.2277, -0.2507,  0.0462],
        [ 0.3015, -0.1660,  0.2494,  0.2973, -0.1908, -0.3775,  0.1546],
        [ 0.2464,  0.1836,  0.0328,  0.3602, -0.2848,  0.3500, -0.0623],
        [ 0.1666, -0.0877, -0.3480,  0.1254,  0.2744,  0.1182,  0.0087],
        [ 0.1682,  0.0576, -0.3607,  0.0985,  0.2092, -0.2570, -0.0953],
        [ 0.2103, -0.1325, -0.3326, -0.2916,  0.0700, -0.2710,  0.3406],
        [-0.2629, -0.3371,  0.2206, -0.2819,  0.3126, -0.0363,  0.1584]])


In [ ]:
EPOCHS = 300
load_index = 0

train(EPOCHS, load_index)

Epoch: 001, Model Loss: 0.7860 Accuracy: 0.5053191489361702
Epoch: 002, Model Loss: 0.6709 Accuracy: 0.6542553191489362
Epoch: 003, Model Loss: 0.6672 Accuracy: 0.6648936170212766
Epoch: 004, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 005, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 006, Model Loss: 0.6662 Accuracy: 0.6648936170212766
Epoch: 007, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 008, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 009, Model Loss: 0.6657 Accuracy: 0.6648936170212766
Epoch: 010, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 011, Model Loss: 0.6657 Accuracy: 0.6648936170212766
Epoch: 012, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 013, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 014, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 015, Model Loss: 0.6658 Accuracy: 0.6648936170212766
Epoch: 016, Model Loss: 0.6660 Accuracy: 0.6648936170212766
Epoch: 017, Model Loss: 0.6658 Accuracy:

In [ ]:
print(gin_model_example.state_dict()['gin_mlp_layers.0.gin_mlp_layers.0.weight'])

tensor([[ 2.2462e-01,  5.6337e-02, -1.1318e-01, -1.8438e-03,  3.2128e-40,
          2.2555e-03,  4.0256e-03],
        [-4.3071e-01, -2.0260e-02,  6.0687e-02,  2.1374e-03,  1.9888e-38,
         -2.0845e-03,  1.2141e-06],
        [ 4.3454e-01, -5.4688e-01,  2.0320e-01, -4.8693e-04, -1.3865e-38,
          4.9105e-02, -9.2754e-06],
        [ 7.3214e-01,  1.0336e-01, -6.8947e-02, -9.9545e-05, -4.8786e-40,
          6.3891e-05,  1.2414e-01],
        [-2.1561e-01, -3.4010e-01, -1.4675e-01,  1.3376e-05,  3.1561e-40,
          2.5296e-02, -9.2010e-06],
        [-1.5141e-01, -6.7731e-02,  9.4892e-02, -1.1764e-02,  7.6239e-40,
          3.0364e-02, -4.1464e-06],
        [-7.8182e-02, -3.4878e-02,  2.9040e-01, -1.2821e-02,  3.7489e-40,
          1.4177e-01, -2.1827e-05]])


In [ ]:
'''
from torch_geometric.utils import dropout
from torch_geometric.loader import DataLoader
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch_geometric.utils.convert import to_scipy_sparse_matrix
from torch_geometric.utils.train_test_split_edges import torch_geometric
import networkx as nx
import numpy as np
from torch_geometric.nn import GCNConv
import sys
from torch_geometric.datasets import TUDataset
from torch_geometric.nn import global_add_pool
from scipy.sparse import csr_matrix
py_path = '/content/drive/MyDrive/Explainability Methods/Models/Script/Layers/'
sys.path.insert(0,py_path)
import GIN_MLP_Layers as gin_mlp_layers


class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GIN_Model(nn.Module):
    def __init__(self, num_mlp_layers, Bias, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, num_classes, dropout_rate, Weight_Initializer):
        super(GIN_Model, self).__init__()

        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.mlp_act_fun = mlp_act_fun
        self.lin_act_fun = mlp_act_fun
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.Weight_Initializer = Weight_Initializer

        self.num_mlp_layers = num_mlp_layers
        self.Bias = Bias

        self.eps = nn.Parameter(torch.zeros(self.num_mlp_layers))
        self.gin_mlp_layers = nn.ModuleList()
        self.global_summing = GlobalSUMPool()

        self.lin1 = nn.Linear(in_features=self.mlp_output_dim * (self.num_mlp_layers + 1), out_features=self.mlp_input_dim * (self.num_mlp_layers + 1))
        self.lin2 = nn.Linear(in_features=self.mlp_input_dim * (self.num_mlp_layers + 1), out_features=self.num_classes)
        self.dorpout = nn.Dropout(p=dropout_rate)
        self.act_fun_softmax = F.softmax

        for i in range(self.num_mlp_layers):
            self.gin_mlp_layers.append(gin_mlp_layers.GIN_MLPs(num_slp_layers=self.num_slp_layers, mlp_input_dim=self.mlp_input_dim, mlp_hid_dim=self.mlp_hid_dim, mlp_output_dim=self.mlp_output_dim, mlp_act_fun=self.mlp_act_fun, Bias=self.Bias))

        if self.lin_act_fun == 'ReLu':
            self.lin_act_fun = F.relu
            #print('ReLu is Selected.')
        elif self.lin_act_fun == 'eLu':
            self.lin_act_fun = nn.functional.elu
            #print('eLu is Selected.')
        elif self.lin_act_fun == 'tanh':
            self.lin_act_fun = torch.tanh
            #print('tanh is Selected.')

        mean = 0
        std = 0.1
        self.initialize_weights(self.Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.xavier_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.xavier_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.kaiming_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.kaiming_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.normal_(modules.weight, mean=mean, std=std)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass


    def gin_neighborhood_aggregation(self, h, batched_graphs, edge_mask):

        #joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())# + torch.eye(len(torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())))
        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)
        if batched_graphs.batch == None:
            batch_size = 1
        else:
            batch_size = batched_graphs.num_graphs

        pooled = torch.mm(joint_tilda_adjacency_matrix, batched_graphs.x)


        return joint_tilda_adjacency_matrix, pooled


    def gin_layer_process_eps(self, h, layer, batched_graphs, edge_mask):

        joint_tilda_adjacency_matrix, pooled = self.gin_neighborhood_aggregation(h, batched_graphs, edge_mask)

        pooled = pooled + (1 + self.eps[layer])*h
        pooled_rep = self.gin_mlp_layers[layer](pooled)


        return pooled_rep

    def merging_process(self, one_mlp, graph_sizes):
        new=[]
        start=0
        for j in range(len(graph_sizes)):
            end = start + graph_sizes[j]
            new.append(one_mlp[start:end])
            start = end
        return new

    def reshape_mlps_outputs(self, mlps_output_embeds, graph_sizes):
        merged_mlps_output_embeds = []
        for i in range(len(graph_sizes)):
            merged_mlps_output_embeds.append([])

        for i in range(len(mlps_output_embeds)):
            for j in range(len(mlps_output_embeds[i])):
                merged_mlps_output_embeds[j].extend(mlps_output_embeds[i][j])
        return merged_mlps_output_embeds





    def forward(self, batched_graphs, edge_mask):
        #X_concatinated = [graph for graph in batched_graphs.x]
        #X_concatinated = torch.stack(X_concatinated, dim=0)

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
        else:
            graph_sizes = [len(batched_graphs.x)]

        mlps_output_embeds = []
        mlps_output_embeds.append(batched_graphs.x)
        hid_rep = batched_graphs.x

        for layer in range(self.num_mlp_layers):
            hid_rep = self.gin_layer_process_eps(hid_rep, layer, batched_graphs, edge_mask)
            mlps_output_embeds.append(hid_rep)
        #print("the last mlp's: ",h.size())
        #h = torch.split(hid_rep, graph_sizes)


        mlps_output_embeds_stacked = torch.stack(mlps_output_embeds)

        mlp_outputs_globalSUMpooled = self.global_summing(mlps_output_embeds_stacked, batched_graphs.batch)

        #merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlp_outputs_globalSUMpooled, graph_sizes)
        if batched_graphs.batch == None:
            #merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlps_output_embeds, graph_sizes, batched_graphs.batch)
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(torch.unsqueeze(mlp_outputs_globalSUMpooled, dim=1), graph_sizes)
        else:
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlp_outputs_globalSUMpooled, graph_sizes)
        lin1_output = self.lin1(torch.tensor(merged_mlps_output_embeds_reshaped))
        lin1_output = self.lin_act_fun(lin1_output)

        lin1_output_dropouted = self.dorpout(lin1_output)

        lin2_output = self.lin2(lin1_output_dropouted)
        lin2_output_softmaxed = self.act_fun_softmax(lin2_output, dim=1)

        return mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed



#dataset = TUDataset(root='data/TUDataset', name='MUTAG')
#batch_size = 20
#node_feat_size = len(dataset[0].x[0])
#batched_dataset = DataLoader(dataset, batch_size=batch_size, shuffle=False)
#gin_model_example = GIN_Model(num_mlp_layers=4, Bias=True, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=7, mlp_output_dim=7, #mlp_act_fun="ReLu", num_classes=2, dropout_rate=0.5, Weight_Initializer=3)
#print(gin_model_example)

#for batched_graphs in batched_dataset:
#    #x, edge_index, batch, y = batched_graphs.x, batched_graphs.edge_index, batched_graphs.batch, batched_graphs.y
#    mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, #lin1_output_dropouted, lin2_output, lin2_output_softmaxed = gin_model_example(batched_graphs)
#    print("lin2_output_softmaxed: ", lin2_output_softmaxed.size())
#    print(lin2_output_softmaxed)
#    break'''

GIN_Model(
  (gin_mlp_layers): ModuleList(
    (0-3): 4 x GIN_MLPs(
      (gin_mlp_layers): ModuleList(
        (0-1): 2 x Linear(in_features=7, out_features=7, bias=True)
      )
      (gin_batch_normalization): ModuleList(
        (0): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (global_summing): GlobalSUMPool()
  (lin1): Linear(in_features=35, out_features=35, bias=True)
  (lin2): Linear(in_features=35, out_features=2, bias=True)
  (dorpout): Dropout(p=0.5, inplace=False)
)


In [ ]:
'''from IPython.core.display import deepcopy
class GlobalSUMPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return torch_geometric.nn.global_add_pool(x, batch)
################################################################################
class IdenticalPool(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, x, batch):
        return x

################################################################################
class GIN_Model(nn.Module):
    def __init__(self, num_mlp_layers, Bias, num_slp_layers, mlp_input_dim, mlp_hid_dim, mlp_output_dim, mlp_act_fun, num_classes,
                 dropout_rate, Weight_Initializer):
        super(GIN_Model, self).__init__()

        self.mlp_input_dim = mlp_input_dim
        self.mlp_hid_dim = mlp_hid_dim
        self.mlp_output_dim = mlp_output_dim
        self.num_slp_layers = num_slp_layers
        self.mlp_act_fun = mlp_act_fun
        self.lin_act_fun = mlp_act_fun
        self.num_classes = num_classes
        self.dropout_rate = dropout_rate
        self.Weight_Initializer = Weight_Initializer

        self.num_mlp_layers = num_mlp_layers
        self.Bias = Bias

        self.eps = nn.Parameter(torch.zeros(self.num_mlp_layers))
        self.gin_mlp_layers = nn.ModuleList()
        self.global_summing = GlobalSUMPool()

        self.lin1 = nn.Linear(in_features=self.mlp_output_dim * (self.num_mlp_layers + 1), out_features=self.mlp_input_dim * (self.num_mlp_layers + 1))
        self.lin2 = nn.Linear(in_features=self.mlp_input_dim * (self.num_mlp_layers + 1), out_features=self.num_classes)
        self.dorpout = nn.Dropout(p=dropout_rate)
        self.act_fun_softmax = F.softmax

        for i in range(self.num_mlp_layers):
            self.gin_mlp_layers.append(GIN_MLPs(num_slp_layers=self.num_slp_layers, mlp_input_dim=self.mlp_input_dim,
                                                               mlp_hid_dim=self.mlp_hid_dim, mlp_output_dim=self.mlp_output_dim,
                                                               mlp_act_fun=self.mlp_act_fun, Bias=self.Bias))

        if self.lin_act_fun == 'ReLu':
            self.lin_act_fun = F.relu
            #print('ReLu is Selected.')
        elif self.lin_act_fun == 'eLu':
            self.lin_act_fun = nn.functional.elu
            #print('eLu is Selected.')
        elif self.lin_act_fun == 'tanh':
            self.lin_act_fun = torch.tanh
            #print('tanh is Selected.')

        mean = 0
        std = 0.1
        self.initialize_weights(self.Weight_Initializer, Bias, mean, std)

    def initialize_weights(model, Weight_Initializer, Bias, mean, std):
        # 1. Xavier Normal_.  2. Kaiming Normal_.  3. Uniform (0,0.1std)
        if Weight_Initializer == 1:                                             #.      1. Xavier Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.xavier_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.xavier_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 2:                                             #.      2. Kaiming Normal_.
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.kaiming_normal_(layers.weight)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.kaiming_normal_(modules.weight)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass

        if Weight_Initializer == 3:                                             #.      3. Uniform (0,0.1std)
            for i, modules in enumerate(model.children()):
                if isinstance(modules, torch.nn.ModuleList):
                    for module in modules:
                        if isinstance(module, gin_mlp_layers.GIN_MLPs):
                            for final_module in module.children():
                                if isinstance(final_module, torch.nn.ModuleList):
                                    for layers in final_module:
                                        if isinstance(layers, torch.nn.Linear):
                                            torch.nn.init.normal_(layers.weight, mean=mean, std=std)
                                            #layers.weight.data.zero_()
                                            #print(layers.weight)
                                            if Bias:
                                                layers.bias.data.zero_()
                                                #print("ok")
                                        elif isinstance(layers, torch.nn.BatchNorm1d):
                                            #print("ok")
                                            pass
                elif isinstance(modules, torch.nn.Linear):
                    #print("predict layer")
                    #modules.weight.data.zero_()
                    #print(modules.weight)
                    torch.nn.init.normal_(modules.weight, mean=mean, std=std)
                elif isinstance(modules, GlobalSUMPool):
                    #print("GlobalSUMPool")
                    #print(modules)
                    pass
                elif isinstance(modules, nn.Dropout):
                    #print("Dropout")
                    #print(modules)
                    pass
                else:
                    pass


    def gin_neighborhood_aggregation(self, new_adjacecny, new_features):

        pooled = torch.bmm(new_adjacecny, new_features)


        return pooled


    def gin_layer_process_eps(self, hid_rep, layer, new_adjacecny, new_features):

        pooled = self.gin_neighborhood_aggregation(new_adjacecny, new_features)


        pooled = pooled + (1 + self.eps[layer])*hid_rep
        pooled_rep = self.gin_mlp_layers[layer](pooled)


        return pooled_rep

    def merging_process(self, one_mlp, graph_sizes):
        new=[]
        start=0
        for j in range(len(graph_sizes)):
            end = start + graph_sizes[j]
            new.append(one_mlp[start:end])
            start = end
        return new

    def reshape_mlps_outputs(self, mlps_output_embeds, graph_sizes):
        merged_mlps_output_embeds = []
        for i in range(len(graph_sizes)):
            merged_mlps_output_embeds.append([])

        for i in range(len(mlps_output_embeds)):
            for j in range(len(mlps_output_embeds[i])):
                merged_mlps_output_embeds[j].extend(mlps_output_embeds[i][j])
        return merged_mlps_output_embeds


    def computational_matrix(self, batched_graphs, edge_mask):

        if edge_mask == None:
            joint_tilda_adjacency_matrix = torch.tensor(to_scipy_sparse_matrix(batched_graphs.edge_index).todense())
        else:
            joint_tilda_adjacency_matrix = torch.tensor(csr_matrix((np.array(edge_mask), (np.array(batched_graphs.edge_index[0]), np.array(batched_graphs.edge_index[1])))).todense())

        joint_tilda_adjacency_matrix = joint_tilda_adjacency_matrix.type(torch.float32)


        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
            batch_size = batched_graphs.num_graphs
        else:
            graph_sizes = [len(batched_graphs.x)]
            batch_size = 1
        max_number_of_nodes_in_batch_of_graphs = max(graph_sizes)


        adjacency_list = []
        feature_list = []
        for i in range(batch_size):
            start = i * graph_sizes[i]
            end = (i + 1) * graph_sizes[i]
            un_padded_adj = joint_tilda_adjacency_matrix[start:end, start:end]
            off_set = max_number_of_nodes_in_batch_of_graphs - un_padded_adj.size()[0]
            if un_padded_adj.size()[0] <= max_number_of_nodes_in_batch_of_graphs:
                un_padded_adj = F.pad(un_padded_adj, (0, off_set, 0, off_set), mode='constant', value=0)
                un_padded_adj = un_padded_adj.type(torch.float32)

            un_padded_adj = un_padded_adj.type(torch.float32)
            adjacency_list.append(un_padded_adj)

            un_padded_feat = batched_graphs.x[start:end, :]
            un_padded_feat = F.pad(un_padded_feat, (0, 0, 0, off_set), mode='constant', value=0)
            un_padded_feat = un_padded_feat.type(torch.float32)
            un_padded_feat.require_grad = True
            feature_list.append(un_padded_feat)

        adjacency_list = list(map(lambda x: torch.unsqueeze(x, 0), adjacency_list))
        feature_list = list(map(lambda x: torch.unsqueeze(x, 0), feature_list))

        new_adjacecny = torch.cat(adjacency_list, dim=0)
        new_features = torch.cat(feature_list, dim=0)

        return new_adjacecny, new_features


    def forward(self, batched_graphs, edge_mask):

        if batched_graphs.batch is not None:
            graph_sizes = [len(batched_graphs[i].x) for i in range(len(batched_graphs))]
        else:
            graph_sizes = [len(batched_graphs.x)]
        batched_graphs2 = deepcopy(batched_graphs)
        print(batched_graphs.batch, batched_graphs.batch.size())
        new_adjacecny, new_features = self.computational_matrix(batched_graphs, edge_mask)
        mlps_output_embeds = []
        mlps_output_embeds.append(new_features)



        source = new_features
        for layer in range(self.num_mlp_layers):
            new_features = self.gin_layer_process_eps(new_features, layer, new_adjacecny, new_features)
            source = torch.cat((source, new_features),0)
            mlps_output_embeds.append(new_features)

        batch_list = []
        for i in range(source.size()[0]):
            for j in range(source.size()[1]):
                batch_list.append(i)
        batch_np = np.array(batch_list)
        batch_torch = torch.from_numpy(batch_np)
        source = source.view(source.size()[0]*source.size()[1], source.size()[2])
        print("here", source.size())
        print("new_features: ", new_features.size())


        #mlps_output_embeds_stacked = torch.stack(mlps_output_embeds)
        #print("mlps_output_embeds_stacked: ", mlps_output_embeds_stacked.size())

        mlp_outputs_globalSUMpooled = self.global_summing(source, batch_torch)
        print("mlp_outputs_globalSUMpooled: ", mlp_outputs_globalSUMpooled.size())

        if batched_graphs.batch == None:
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(torch.unsqueeze(mlp_outputs_globalSUMpooled, dim=1), graph_sizes)
        else:
            merged_mlps_output_embeds_reshaped = self.reshape_mlps_outputs(mlp_outputs_globalSUMpooled, graph_sizes)
        lin1_output = self.lin1(torch.tensor(merged_mlps_output_embeds_reshaped))
        lin1_output = self.lin_act_fun(lin1_output)

        lin1_output_dropouted = self.dorpout(lin1_output)

        lin2_output = self.lin2(lin1_output_dropouted)
        lin2_output_softmaxed = self.act_fun_softmax(lin2_output, dim=1)

        return mlps_output_embeds, mlps_output_embeds_stacked, mlp_outputs_globalSUMpooled, merged_mlps_output_embeds_reshaped, lin1_output, lin1_output_dropouted, lin2_output, lin2_output_softmaxed
node_feat_size=7
gin_model_example = GIN_Model(num_mlp_layers=4, Bias=True, num_slp_layers=2, mlp_input_dim=node_feat_size, mlp_hid_dim=7,
                              mlp_output_dim=7, mlp_act_fun="ReLu", num_classes=2, dropout_rate=0.5, Weight_Initializer=3)
#print(gin_model_example)

'''